<a href="https://colab.research.google.com/github/LETIMEI/Portfolio-Leti-/blob/main/RAG%20Architecture%20Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Augmentation Generation: BBC Politics News

##Packages Installation and Import

In [ ]:
# Install Ollama v0.1.30
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10091    0 10091    0     0  42341      0 --:--:-- --:--:-- --:--:-- 42221
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
%%capture
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

# Add the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # print the global variable to check it saved

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

time.sleep(5)  # Makes Python wait for 5 seconds

# Install prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install llama_index.readers.web
!pip install llama-index-vector-stores-chroma
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SummaryIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

# Use the global variable (OLLAMA_MODEL) as our LLM
# Set a timeout of 8 minutes in case of CPU
llm = Ollama(model=OLLAMA_MODEL, request_timeout=480.0)

In [ ]:
# Query the model via the command line
# First time running it will "pull" (import) the model

# Test question 1: simple question

!ollama run $OLLAMA_MODEL "Who is Neil Kinnock?"

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b.

In [ ]:
!ollama run $OLLAMA_MODEL "Why could CSA close?"

In [ ]:
%%capture

# Install prerequisites
!pip install datasets
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

In [ ]:
# use capture to hide output messages
%%capture

!pip install accelerate -U
!pip install -U sentence-transformers
!pip install faiss-gpu
!pip install arxiv

import faiss
import arxiv
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE

In [ ]:
%%capture
!pip install llama_index.core
!pip install llama_index.readers.file
!pip install llama_index.readers.web

##Data Loading

BBC News data

In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("RealTimeData/News_August_2023")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5059 [00:00<?, ? examples/s]

In [ ]:
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Display the DataFrame
print(df.head())

,authors,date_download,date_modify,date_publish,description,filename,image_url,language,localpath,maintext,source_domain,title,title_page,title_rss,url
0,[],2023-08-01 01:20:55+00:00,_,2023-08-01 01:10:02,A consultant cardiologist at the Federal Medic...,https%3A%2F%2Fdailytrust.com%2Ftherapeutic-lif...,https://dailytrust.com/wp-content/uploads/2018...,en,_,A consultant cardiologist at the Federal Medic...,dailytrust.com,‘Therapeutic lifestyle modification’ lowers ri...,_,_,https://dailytrust.com/therapeutic-lifestyle-m...
1,[],2023-08-01 01:20:06+00:00,_,2023-08-01 01:13:54,The Nasarawa State government is taking measur...,https%3A%2F%2Fdailytrust.com%2Fhow-nasarawa-go...,https://dailytrust.com/wp-content/uploads/2022...,en,_,The Nasarawa State government is taking measur...,dailytrust.com,How Nasarawa govt is responding to diphtheria ...,_,_,https://dailytrust.com/how-nasarawa-govt-is-re...
2,[],2023-08-01 01:20:19+00:00,_,2023-08-01 01:07:57,Lawyers are divided over the renewed moves to ...,https%3A%2F%2Fdailytrust.com%2Fnba-conference-...,https://dailytrust.com/wp-content/uploads/2022...,en,_,Lawyers are divided over the renewed moves to ...,dailytrust.com,NBA Conference: Lawyers divided over parallel ...,_,_,https://dailytrust.com/nba-conference-lawyers-...
3,[],2023-08-01 01:20:00+00:00,_,2023-08-01 00:37:29,D’Tigress will face the winners between Mozamb...,https%3A%2F%2Fdailytrust.com%2Fdtigress-to-fac...,https://dailytrust.com/wp-content/uploads/2022...,en,_,D’Tigress will face the winners between Mozamb...,dailytrust.com,D’Tigress to face Mozambique or Cote d’Ivoire ...,_,_,https://dailytrust.com/dtigress-to-face-mozamb...
4,[],2023-08-01 01:20:37+00:00,_,2023-08-01 01:11:50,Liver cancer patients are being spared overnig...,https%3A%2F%2Fdailytrust.com%2Fradioactive-bea...,https://dailytrust.com/wp-content/uploads/2022...,en,_,Liver cancer patients are being spared overnig...,dailytrust.com,Radioactive beads in the wrist that can fight ...,_,_,https://dailytrust.com/radioactive-beads-in-th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5054,"[Carrie Young, Natalie Herbick]",2023-08-01 01:26:26+00:00,_,2023-08-01 01:01:07,She was diagnosed with stage 2B HER2-POSITIVE ...,https%3A%2F%2Fwww.wkbn.com%2Fnews%2Fohio%2Fbre...,https://www.wkbn.com/wp-content/uploads/sites/...,en,_,CLEVELAND (WJW) – Lesley Kiraly Hosta was just...,www.wkbn.com,"Breast cancer survivor says research, newer dr...",_,_,https://www.wkbn.com/news/ohio/breast-cancer-s...
5055,[Brooke Williams],2023-08-01 01:26:20+00:00,_,2023-08-01 00:19:52,Country artist Luke Bryan invited a local girl...,https%3A%2F%2Fwww.wkbn.com%2Fnews%2Fnational-w...,https://www.wkbn.com/wp-content/uploads/sites/...,en,_,DENVER (KDVR) — Country artist Luke Bryan invi...,www.wkbn.com,Child with cancer gets invited back stage to L...,_,_,https://www.wkbn.com/news/national-world/luke-...
5056,[Stephanie Whiteside],2023-08-01 01:04:45+00:00,_,2023-07-31 23:15:01,Social media has gone wild as people claim the...,https%3A%2F%2Ffox2now.com%2Fnews%2Fnational%2F...,https://fox2now.com/wp-content/uploads/sites/1...,en,_,"(NewsNation) — As in decades past, the questio...",fox2now.com,Did the government confirm aliens exist?,_,_,https://fox2now.com/news/national/did-the-gove...
5057,[Brooke Williams],2023-08-01 01:04:51+00:00,_,2023-08-01 00:18:40,Country artist Luke Bryan invited a local girl...,https%3A%2F%2Ffox2now.com%2Fnews%2Fnational%2F...,https://fox2now.com/wp-content/uploads/sites/1...,en,_,DENVER (KDVR) — Country artist Luke Bryan invi...,fox2now.com,Child with cancer gets invited back stage to L...,_,_,https://fox2now.com/news/national/luke-bryan-i...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
import pandas as pd
import re  # Importing regular expressions library
from google.colab import files
import io

# Create an upload button and get the uploaded file
uploaded_files = files.upload()

# Get the file data from the uploaded files
file_name = next(iter(uploaded_files))  # This gets the name of the first uploaded file
file_data = uploaded_files[file_name]

# Read the CSV data into a DataFrame
df = pd.read_csv(io.BytesIO(file_data))

# Only keep politics data for further exploration
#df = df[df['labels'] == 'politics']

#df = df[df['labels'].isin(['politics', 'tech'])]

# Remove duplicate entries in 'data' column
df = df.drop_duplicates(subset=['Article text'])

# Display the filtered DataFrame
df.head()


Saving CNN_Articels_clean.csv to CNN_Articels_clean.csv


,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...


In [ ]:
df

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...
...,...,...,...,...,...,...,...,...,...,...,...
4071,4725,Story by Reuters,2021-12-01 10:01:29,sport,tennis,https://www.cnn.com/2021/12/01/tennis/australi...,Australian Open: Australia's vaccine mandate i...,The Australian Open's Covid-19 vaccine mandate...,"tennis, Australian Open: Australia's vaccine m...",Australia's vaccine mandate is not to 'blackma...,(Reuters)The Australian Open's Covid-19 vacci...
4072,4726,"Homero De la Fuente, CNN",2021-12-01 17:56:15,sport,golf,https://www.cnn.com/2021/12/01/golf/south-afri...,Four golfers test positive ahead of South Afri...,Four golfers scheduled to play in the South Af...,"golf, Four golfers test positive ahead of Sout...",Four golfers test positive ahead of South Afri...,(CNN)Four golfers scheduled to play in the So...
4073,4727,"Amy Woodyatt, CNN",2021-12-01 11:32:15,sport,tennis,https://www.cnn.com/2021/12/01/tennis/peng-shu...,Peng Shuai: 'Unanimous conclusion' that tennis...,Long-time International Olympic Committee (IOC...,"tennis, Peng Shuai: 'Unanimous conclusion' tha...",'Unanimous conclusion' that Peng Shuai is 'fin...,(CNN)Long-time International Olympic Committe...
4074,4728,Story by Reuters,2021-12-01 17:27:34,news,europe,https://www.cnn.com/2021/12/01/europe/cow-dung...,"This company is ""zapping"" cow dung with lightn...",A Norwegian technology company has found a way...,"europe, This company is ""zapping"" cow dung wit...",Zapping cow dung with lightning is helping to ...,A Norwegian technology company has found a way...


In [ ]:
#store each row in column 'data' in separated txt files

!mkdir -p '/content/bbc_data/' # create an empty directory called "bbc_data"

count = 0

for index, row in df.iterrows():
    data_content = row['data']
    fname = "/content/bbc_data/Output" + str(count) + ".txt"
    with open(fname, "w") as text_file:
        text_file.write(data_content)
    count += 1


##Chunking

###Semantic Splitter

In [ ]:
# Load documents
reader = SimpleDirectoryReader("/content/bbc_data") # load documents from the /data folder
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
#embed_model = HuggingFaceEmbedding(model_name="TaylorAI/gte-tiny")
llm = Ollama(model=OLLAMA_MODEL, request_timeout=1500.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 417 docs


In [ ]:
# run this one, works
from llama_index.core.node_parser import SemanticSplitterNodeParser
from pathlib import Path  # for finding the file

parser = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=90, embed_model=embed_model
)

semantic_nodes = parser.get_nodes_from_documents(docs)

# Here, semantic_nodes will contain the output which can be used for further processing
print(semantic_nodes)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
semantic_nodes

[TextNode(id_='ebfe7bd0-d4dc-460e-bff9-192945e0988f', embedding=None, metadata={'file_path': '/content/bbc_data/Output0.txt', 'file_name': 'Output0.txt', 'file_type': 'text/plain', 'file_size': 2499, 'creation_date': '2024-05-18', 'last_modified_date': '2024-05-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5b3e75ab-d48e-449f-aeeb-5b96e959e0af', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/bbc_data/Output0.txt', 'file_name': 'Output0.txt', 'file_type': 'text/plain', 'file_size': 2499, 'creation_date': '2024-05-18', 'last_modified_date': '2024-05-18'}, hash='f263f950a16ee2f5f42654bffdfca69ef6783826faf9cd88a5961c5e85289465'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_

In [ ]:
semantic_nodes[0].text

'Baron Kinnock makes Lords debut  Former Labour leader Neil Kinnock has officially been made a life peer during a ceremony in the House of Lords.  He will be known Baron Kinnock of Bedwellty - after his former constituency. Lord Kinnock - who led Labour from 1983 until 1992 - was until recently one of Britains EU commissioners. A former critic of the House of Lords, he has said he will use the Upper House to advocate its reform and to talk on issues like higher education. "I accepted the kind invitation to enter the House of Lords as a working peer for practical political reasons," he said when his peerage was first announced. "It is a good base for campaigning on national issues like education, sustainable transport, industrial change and the ageing society and global concerns, particularly poverty and oppression." During his induction into the Upper House, Lord Kinnock was accompanied by Lords Leader Baroness Amos and Baroness Royall of Blaisdon, a former aide to the ex-Labour leader

In [ ]:
# extract splitted text from the semantic output

# Assuming 'semantic_nodes' is your list of TextNode objects
all_texts = [node.text for node in semantic_nodes]

# Now 'all_texts' contains all the extracted texts from each TextNode
all_texts

['Baron Kinnock makes Lords debut  Former Labour leader Neil Kinnock has officially been made a life peer during a ceremony in the House of Lords.  He will be known Baron Kinnock of Bedwellty - after his former constituency. Lord Kinnock - who led Labour from 1983 until 1992 - was until recently one of Britains EU commissioners. A former critic of the House of Lords, he has said he will use the Upper House to advocate its reform and to talk on issues like higher education. "I accepted the kind invitation to enter the House of Lords as a working peer for practical political reasons," he said when his peerage was first announced. "It is a good base for campaigning on national issues like education, sustainable transport, industrial change and the ageing society and global concerns, particularly poverty and oppression." During his induction into the Upper House, Lord Kinnock was accompanied by Lords Leader Baroness Amos and Baroness Royall of Blaisdon, a former aide to the ex-Labour leade

In [ ]:
!mkdir -p '/content/splitted_bbc_politics_data/' # create an empty directory called "data"

count = 0

for doc in all_texts: # iterate through the results
  fname = "/content/splitted_bbc_politics_data/Output" + str(count) + ".txt"
  with open(fname, "w") as text_file:
    text_file.write(doc) # save the file
  count += 1 # increment the count

##Embedding
Now we have our data we will create embedings of the abstracts (encoding) using sentence level [DistilBERT](https://huggingface.co/docs/transformers/en/model_doc/distilbert). DistilBERT is a smaller version of classic BERT, designed to have similar performance with 40% fewer parameters (so faster).

In [ ]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Convert abstracts to vectors
embeddings = model.encode(df.abstract.to_list(), show_progress_bar=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

AttributeError: 'DataFrame' object has no attribute 'abstract'

##Vector Database Setup

In [ ]:
# bbc data

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

#embed_model = HuggingFaceEmbedding(model_name="TaylorAI/gte-tiny")
llm = Ollama(model=OLLAMA_MODEL, request_timeout=1500.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

# Load documents
reader = SimpleDirectoryReader("/content/splitted_bbc_politics_data") # load documents from the /data folder
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db")

# Create a collection/table ("demo-for-ram") in the db
chroma_collection = db.create_collection("my-demo")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the vector index
vector_index = VectorStoreIndex.from_documents(
    docs, # the file created earlier
    storage_context = storage_context,
    embed_model = embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

Loaded 1397 docs
name='my-demo3' id=UUID('f6d73a74-5660-49fc-b316-229bf8af1b65') metadata=None tenant='default_tenant' database='default_database'
Collection name is: my-demo3


##Prompt Template Setup

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

qa_prompt_str = (
    "Below is the context information.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Please just say 'I don't know the answer' if the answer is not provided in the context."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

##Query Testing

In [ ]:
#ignore this one
query_engine = vector_index.as_query_engine(response_mode="refine") # use the vector db for queries
response = query_engine.query("Who is Neil Kinnock?") # query Phi-3 with context
response.response # print the response

' Neil Kinnock is a former British politician who served as the leader of the Labour Party from 1993 to 1997. He was born in Tredegar, South Wales in 1942 and went on to become an influential figure in British politics. In recent years, he has been advocating for reform of the House of Lords and expressing his views on important political issues such as higher education, sustainable transport, industrial change, and the ageing society.\n'

In [ ]:
response.metadata

{'ef1adacd-506f-44d6-8fac-fa8ddf9db24a': {'file_path': '/content/splitted_bbc_politics_data/Output0.txt',
  'file_name': 'Output0.txt',
  'file_type': 'text/plain',
  'file_size': 1378,
  'creation_date': '2024-05-18',
  'last_modified_date': '2024-05-18'},
 'bad518a5-6d3f-479d-b8b0-d6d3166a5903': {'file_path': '/content/splitted_bbc_politics_data/Output1382.txt',
  'file_name': 'Output1382.txt',
  'file_type': 'text/plain',
  'file_size': 450,
  'creation_date': '2024-05-18',
  'last_modified_date': '2024-05-18'}}

In [ ]:
# Test1: Answerable question (politics)
print(
    vector_index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("Why could CSA close?")
)

 The Child Support Agency (CSA) may close because it has failed to improve its service and is facing criticism from a report by the Commons work and pensions committee.



In [ ]:
# Test2: Not answerable question
print(
    vector_index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("What are the most famous plays written by William Shakespeare?")
)

 I do not have access to the specific list of plays written by william shakespeare. however, some of his most famous plays include "romeo and juliet," "hamlet," "macbeth," "a Midsummer Night's Dream," "poetry," and many more.



In [ ]:
# Test3: Complex question
print(
    vector_index.as_query_engine(
        response_mode="refine",
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("What are the main issues that the Muslim Association of Britain (MAB) believes Muslims should consider before voting in the next general election?")
)

 The Muslim Association of Britain (MAB) believes there are several main issues that should be considered by Muslim voters before casting their votes in the next general election. These include: 
1. The war on Iraq, which is expected to have a significant impact due to its influence on voting intentions and the number of seats Muslims could potentially sway. 
2. The Palestinian situation, an issue that holds importance for many Muslim voters and may affect their decision-making process. 
3. The erosion of civil liberties for Muslims in the United Kingdom, which is another factor worth considering while making informed choices during the election. 
4. Economic, social, and educational problems faced by the UK as a whole, regardless of one's religious background. It is crucial to be aware of these issues when casting votes to shape the future of the nation.

